# CVaR optimization benchmark results
This notebook gives an example of how you can replicate the long-only cash portfolio optimization, both in the prior and posterior case, using the fortitudo.tech open-source Python package: https://github.com/fortitudo-tech/fortitudo.tech

These problems are the easiest to solve, so they are a good starting point for understanding the data. After becoming comfortable with these problems, you can proceed to the more challenging ones that include derivatives, leverage constraints, transaction costs and risk budgets.

To see how the results have been computed using Fortitudo Technologies' Investment Analysis module, see the 1_CVaROptBenchmarks notebook: https://github.com/fortitudo-tech/cvar-optimization-benchmarks/blob/main/1_CVaROptBenchmarks.ipynb 

In [1]:
import fortitudo.tech as ft
import numpy as np
import pandas as pd
from cvxopt import matrix
from time import time

In [2]:
# Load data for long-only prior and posterior optimizations
means0 = np.load('means/means0.npy')
res0 = np.load('results/res0.npy')
means12 = np.load('means/means12.npy')
res12 = np.load('results/res12.npy')
q = np.load('data/q.npy')[:, np.newaxis]

# Input data illustration
Below is an illustration of the input data and its dimensions.

Note that we have B=100 expected return samples for the I=10 cash instruments, so the means matrices have dimension (B, I).

For efficient frontier optimization, we use P=9 portfolios, so the results, which are averages over the 100 mean samples, have dimension (I, P), ranging from the lowest to the highest risk portfolio.

In [3]:
# Illustrate shapes
means0.shape, res0.shape, means12.shape, res12.shape

((100, 10), (10, 9), (100, 10), (10, 9))

In [4]:
B, I = means0.shape
_, P = res0.shape
pnl = pd.read_csv('data/pnl_cash.csv')
instruments0_cash = pd.read_csv('data/instruments0_cash.csv', index_col=0)
holding_costs = instruments0_cash['hold'].values
pnl_values = pnl.values
pnl

,DM Gov,Corp IG,Corp HY,EM Gov,DM Equities,EM Equities,Private Equity,Infrastructure,Real Estate,Hedge Funds
0,-0.053715,-0.050922,-0.032982,-0.095512,-0.062872,0.317399,0.099383,0.035433,-0.024514,0.142342
1,0.084538,0.152100,0.293127,0.315402,0.296924,0.602123,0.529613,0.303108,0.354554,0.195189
2,0.049725,0.058961,0.105731,0.102902,0.087244,0.347249,0.366671,-0.004442,0.108793,0.116689
3,0.034873,0.002165,-0.108261,0.056517,-0.185542,-0.041443,-0.227074,0.039706,-0.214388,-0.127768
4,0.000263,0.009671,-0.062047,0.024646,-0.033622,0.149195,-0.084265,0.183832,-0.009315,0.083429
...,...,...,...,...,...,...,...,...,...,...
9995,0.007481,0.003461,0.027986,0.051687,0.073575,0.228842,-0.105466,-0.134338,0.138065,0.112285
9996,0.024132,0.091402,0.128466,0.109805,0.218225,0.188309,0.009244,-0.009969,0.114881,0.182888
9997,0.025901,0.027961,-0.024825,0.124948,0.192028,0.219043,-0.060443,-0.202992,-0.055887,0.060581
9998,0.032867,0.047095,0.088725,-0.051032,-0.035458,-0.085485,0.054598,0.170063,-0.089983,0.078697


In [5]:
instruments0_cash

,hold,buy,sell,mv / exp
DM Gov,0.0023,0.0,0.0,1.0
Corp IG,0.0033,0.0,0.0,1.0
Corp HY,0.0060,0.0,0.0,1.0
EM Gov,0.0044,0.0,0.0,1.0
DM Equities,0.0049,0.0,0.0,1.0
EM Equities,0.0071,0.0,0.0,1.0
Private Equity,0.0356,0.0,0.0,1.0
Infrastructure,0.0152,0.0,0.0,1.0
Real Estate,0.0133,0.0,0.0,1.0
Hedge Funds,0.0171,0.0,0.0,1.0


In [6]:
# Long-only exposure constraints
G = -np.identity(I)
G = np.vstack((G, np.identity(I)))
h = np.zeros(I)
h = np.hstack((h,  np.ones(I)))

In [7]:
ft.cvar_options['demean'] = False
opt = ft.MeanCVaR(pnl_values, G, h, alpha=0.9)  # Uniform probability
opt_q = ft.MeanCVaR(pnl_values, G, h, p=q, alpha=0.9)  # EP posterior probability

In [8]:
start = time()
frontiers = np.full((B, I, P), np.nan)
for b in range(B):
    means = (means0[b] - holding_costs)[np.newaxis, :]
    opt._mean = means
    opt._expected_return_row = matrix(np.hstack((-means, np.zeros((1, 2)))))
    frontiers[b, :, :] = opt.efficient_frontier(P)
end = time()
print(f'{B} efficient frontiers computed in {round(end - start, 2)} seconds.')

100 efficient frontiers computed in 30.27 seconds.


In [9]:
frontiers_q = np.full((B, I, P), np.nan)
for b in range(B):
    means = (means12[b] - holding_costs)[np.newaxis, :]
    opt_q._mean = means
    opt_q._expected_return_row = matrix(np.hstack((-means, np.zeros((1, 2)))))
    frontiers_q[b, :, :] = opt_q.efficient_frontier(P)
end = time()
print(f'{B} efficient frontiers computed in {round(end - start, 2)} seconds.')

100 efficient frontiers computed in 45.85 seconds.


In [10]:
avg_frontier = np.mean(frontiers, axis=0)
avg_frontier_q = np.mean(frontiers_q, axis=0)

In [11]:
display(pd.DataFrame(np.round(avg_frontier, 3), index=instruments0_cash.index))
display(pd.DataFrame(np.round(res0, 3), index=instruments0_cash.index))
print(f'Max difference with uniform probability vector: {np.max(np.abs(avg_frontier - res0))}')

,0,1,2,3,4,5,6,7,8
DM Gov,0.757,0.630,0.485,0.323,0.172,0.062,0.016,0.003,0.00
Corp IG,0.000,0.012,0.035,0.060,0.067,0.042,0.011,0.001,0.00
Corp HY,0.000,0.000,0.000,-0.000,0.000,0.001,0.003,0.001,0.00
EM Gov,0.000,0.007,0.034,0.066,0.100,0.124,0.092,0.030,-0.00
DM Equities,0.000,-0.000,0.001,0.003,0.005,0.006,0.008,0.010,-0.00
EM Equities,0.000,0.002,0.008,0.014,0.022,0.037,0.063,0.099,0.14
Private Equity,0.006,0.062,0.115,0.166,0.220,0.286,0.383,0.524,0.76
Infrastructure,0.042,0.079,0.109,0.139,0.169,0.197,0.220,0.224,0.10
Real Estate,0.071,0.093,0.112,0.132,0.152,0.167,0.161,0.096,0.00
Hedge Funds,0.123,0.113,0.100,0.097,0.094,0.079,0.043,0.012,0.00


,0,1,2,3,4,5,6,7,8
DM Gov,0.757,0.630,0.485,0.323,0.172,0.062,0.016,0.003,0.00
Corp IG,0.000,0.012,0.035,0.060,0.067,0.042,0.011,0.001,-0.00
Corp HY,0.000,0.000,0.000,0.000,0.000,0.001,0.003,0.001,0.00
EM Gov,0.000,0.007,0.034,0.066,0.100,0.124,0.092,0.030,0.00
DM Equities,0.000,0.000,0.002,0.003,0.005,0.006,0.008,0.010,-0.00
EM Equities,0.000,0.002,0.008,0.014,0.022,0.037,0.063,0.099,0.14
Private Equity,0.006,0.062,0.115,0.166,0.220,0.286,0.383,0.524,0.76
Infrastructure,0.042,0.079,0.109,0.139,0.169,0.197,0.220,0.224,0.10
Real Estate,0.071,0.093,0.112,0.132,0.152,0.167,0.161,0.096,-0.00
Hedge Funds,0.123,0.113,0.100,0.097,0.094,0.078,0.043,0.012,0.00


Max difference with uniform probability vector: 4.989563205461123e-05


In [12]:
display(pd.DataFrame(np.round(avg_frontier_q, 3), index=instruments0_cash.index))
display(pd.DataFrame(np.round(res12, 3), index=instruments0_cash.index))
print(f'Max difference with uniform probability vector: {np.max(np.abs(avg_frontier_q - res12))}')

,0,1,2,3,4,5,6,7,8
DM Gov,0.816,0.742,0.593,0.433,0.283,0.160,0.071,0.019,-0.00
Corp IG,0.000,0.014,0.043,0.070,0.080,0.065,0.035,0.011,-0.00
Corp HY,0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.00
EM Gov,0.000,0.006,0.028,0.052,0.079,0.096,0.089,0.053,0.01
DM Equities,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.001,-0.00
EM Equities,0.000,0.004,0.009,0.014,0.019,0.029,0.045,0.068,0.11
Private Equity,0.000,0.040,0.070,0.101,0.135,0.176,0.238,0.340,0.56
Infrastructure,0.031,0.075,0.120,0.164,0.208,0.255,0.298,0.333,0.27
Real Estate,0.074,0.092,0.115,0.141,0.167,0.193,0.204,0.165,0.04
Hedge Funds,0.079,0.028,0.022,0.025,0.027,0.027,0.020,0.011,0.01


,0,1,2,3,4,5,6,7,8
DM Gov,0.816,0.742,0.593,0.433,0.283,0.160,0.071,0.019,0.00
Corp IG,0.000,0.014,0.043,0.070,0.080,0.065,0.035,0.011,-0.00
Corp HY,0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.00
EM Gov,0.000,0.006,0.028,0.052,0.079,0.096,0.089,0.053,0.01
DM Equities,0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.001,-0.00
EM Equities,0.000,0.004,0.009,0.014,0.019,0.029,0.045,0.068,0.11
Private Equity,0.000,0.040,0.070,0.101,0.135,0.176,0.238,0.340,0.56
Infrastructure,0.031,0.075,0.120,0.164,0.208,0.255,0.298,0.333,0.27
Real Estate,0.074,0.092,0.115,0.141,0.168,0.192,0.204,0.165,0.04
Hedge Funds,0.079,0.028,0.022,0.024,0.027,0.027,0.020,0.011,0.01


Max difference with uniform probability vector: 4.9826176690204345e-05
